# Direct Fidelity Estimation example

In [1]:
from pyquil.paulis import ID
from pyquil.gates import X, MEASURE, H, CZ
from pyquil import Program, get_qc
from pyquil.api import get_benchmarker
from forest.benchmarking.direct_fidelity_estimation import ( generate_exhaustive_state_dfe_experiment, 
                                                             generate_exhaustive_process_dfe_experiment,
                                                             acquire_dfe_data,
                                                             estimate_dfe )

In [2]:
# noiseless QVM
qvm = get_qc("9q-generic-qvm", as_qvm=True, noisy=False)

# noisy QVM
noisy_qvm = get_qc("9q-generic-qvm", as_qvm=True, noisy=True)

bm = get_benchmarker()

### State prep (thing we will do DFE on)

In [3]:
p = Program()
prep_prog = p.inst(CZ(0,1))
print(prep_prog)

CZ 0 1



### Get things required for a DFE experiment ... aka experiment object

Namely return a namedtuple consisiting of 
- pauli_in, 
- prog, 
- pauli_out.

In [4]:
# state dfe
state_exp = generate_exhaustive_state_dfe_experiment(prep_prog,[0,1],bm)

# process dfe
process_exp = generate_exhaustive_process_dfe_experiment(prep_prog,[0,1],bm)

In [5]:
print(process_exp.__doc__)


    A tomography-like experiment.

    Many near-term quantum algorithms involve:

     - some limited state preparation
     - enacting a quantum process (like in tomography) or preparing a variational ansatz state
       (like in VQE)
     - measuring observables of the state.

    Where we typically use a large number of (state_prep, measure) pairs but keep the ansatz
    program consistent. This class stores the ansatz program as a :py:class:`~pyquil.Program`
    and maintains a list of :py:class:`ExperimentSetting` objects which each represent a
    (state_prep, measure) pair.

    Settings diagonalized by a shared tensor product basis (TPB) can (optionally) be estimated
    simultaneously. Therefore, this class is backed by a list of list of ExperimentSettings.
    Settings sharing an inner list will be estimated simultaneously. If you don't want this,
    provide a list of length-1-lists. As a convenience, if you pass a 1D list to the constructor
    will expand it to a list of

In [6]:
process_exp

### Acquire DFE data (noiseless)

Returns dfe data 'data' and calibration data 'cal' namedtuple separately

In [7]:
data = acquire_dfe_data(qvm, process_exp)
print("================================")
print(data.pauli_point_est)
print("================================")
print(data.cal_point_est)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Acquire DFE data (noisy QVM)

In [8]:
n_data = acquire_dfe_data(noisy_qvm, process_exp)
print("================================")
print(n_data.pauli_point_est)
print("================================")
print(n_data.cal_point_est)

[0.98029308 0.98001012 0.97378468 0.99085598 0.98838767 1.00748809
 0.97850822 0.98642766 0.9776593  0.98562628 0.98527893 0.98557584
 0.98268496 0.99482669 0.98694985 0.98298629 0.97859327 0.9756962
 0.98036561 0.98685403 0.98770807 0.97604485 0.97252607 0.97601429
 1.00386797 0.98053858 0.99253563 0.98502918 0.99072404 0.99025641
 0.99502878 0.98411885 0.99251531 0.97861805 1.00547445 0.98779385
 0.99682396 1.003861   1.00272727 1.0200052  1.         0.99484536
 0.99229898 0.99437064 0.99477985 0.99774521 1.00384813 1.00175924]
[0.7916 0.7904 0.7858 0.7874 0.8956 0.8814 0.791  0.781  0.7878 0.7792
 0.7744 0.8874 0.8894 0.7732 0.7816 0.7876 0.7848 0.79   0.8862 0.8824
 0.781  0.7848 0.7862 0.7838 0.7756 0.8838 0.8842 0.7882 0.7762 0.78
 0.7644 0.7808 0.8818 0.8886 0.8768 0.8848 0.8816 0.8806 0.88   0.7698
 0.784  0.8924 0.883  0.8882 0.8812 0.887  0.7796 0.7958]


In [9]:
print(n_data.__doc__)

Experimental data from a DFE experiment


### Estimate fidelity (noiseless QVM)

In [12]:
est = estimate_dfe(data,'process')
print("==================================================")
print('Fidelity point estimate is', est.fid_point_est)
print('The standard error of the fidelity point estimate is', est.fid_std_err)
print("==================================================")

Fidelity point estimate is 1.0
The standard error of the fidelity point estimate is 0.0


### Estimate fidelity (noisy QVM)

In [13]:
nest = estimate_dfe(n_data,'process')
print(nest)
print("==================================================")
print('Fidelity point estimate is', nest.fid_point_est)
print('The std error of the fidelity point estimate is', nest.fid_std_err)
print("==================================================")

DFEEstimate(dimension=4, qubits=[0, 1], fid_point_est=0.9915118660759432, fid_std_err=0.0011427903817137557)
Fidelity point estimate is 0.9915118660759432
The std error of the fidelity point estimate is 0.0011427903817137557
